In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

import os

import tensorflow as tf
from tensorflow import keras

In [9]:
data="datas/train"


In [ ]:
datas=ImageDataGenerator(rescale=1/255,validation_split=0.2)

traiM= tf.keras.utils.image_dataset_from_directory(
    directory=data,
    target_size=(100,100),
    batch_size=30,
    class_mode='categorical',
    subset='training',
    seed=42,
)

testM= tf.keras.utils.image_dataset_from_directory(
    directory=data,
    target_size=(100,100),
    batch_size=30,
    class_mode='categorical',
    subset='validation',
    seed=42,
)

Found 13493 images belonging to 33 classes.
Found 3361 images belonging to 33 classes.


In [11]:
model=models.Sequential([
    layers.Conv2D(filters=65,kernel_size=(3,3),activation='relu',input_shape=(100,100,3)),
    layers.MaxPool2D(2,2),
    layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu'),
    layers.MaxPool2D(2,2),
    layers.Dropout(0.5),

    layers.Flatten(),

    layers.Dense(units=64,activation='relu'),
    layers.Dense(units=33,activation='softmax'),
])

In [12]:
model.compile(optimizer='Adam',
               loss=tf.keras.losses.CategoricalCrossentropy(),
                 metrics=['accuracy'],
                   loss_weights=None,
           )

In [ ]:
model.fit(traiM,
        batch_size=None, epochs=5,
        verbose=1, validation_data=testM,
        validation_steps=None,
        validation_freq=1)

Epoch 1/5
450/450 ━━━━━━━━━━━━━━━━━━━━ 71s 154ms/step - accuracy: 0.6876 - loss: 1.1386 - val_accuracy: 1.0000 - val_loss: 0.0024
Epoch 2/5
450/450 ━━━━━━━━━━━━━━━━━━━━ 69s 153ms/step - accuracy: 0.9983 - loss: 0.0086 - val_accuracy: 1.0000 - val_loss: 9.7402e-04
Epoch 3/5
450/450 ━━━━━━━━━━━━━━━━━━━━ 70s 156ms/step - accuracy: 1.0000 - loss: 4.7529e-04 - val_accuracy: 1.0000 - val_loss: 1.0473e-04
Epoch 4/5
450/450 ━━━━━━━━━━━━━━━━━━━━ 71s 157ms/step - accuracy: 0.9999 - loss: 4.0117e-04 - val_accuracy: 0.9125 - val_loss: 0.2461
Epoch 5/5
 25/450 ━━━━━━━━━━━━━━━━━━━━ 1:01 146ms/step - accuracy: 0.9131 - loss: 0.2721

In [ ]:
model.save("model_epoch_10.h5")